## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-07-10-52-44 +0000,nyt,Trump Says Venezuela Will Start Handing Over S...,https://www.nytimes.com/live/2026/01/06/world/...
1,2026-01-07-10-49-51 +0000,nypost,Swiss bar in fire that left 40 dead had missed...,https://nypost.com/2026/01/07/world-news/swiss...
2,2026-01-07-10-48-00 +0000,wsj,Bang & Olufsen CEO Kristian Tear Steps Down,https://www.wsj.com/business/earnings/bang-olu...
3,2026-01-07-10-47-28 +0000,bbc,Why does Trump want Greenland and what could i...,https://www.bbc.com/news/articles/c74x4m71pmjo...
4,2026-01-07-10-46-34 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/07/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,56
1,venezuela,52
5,oil,21
51,new,18
431,maduro,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...,169
48,2026-01-07-08-31-34 +0000,wsj,Stock Market Today: Oil Prices Fall After Trum...,https://www.wsj.com/livecoverage/stock-market-...,164
16,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,164
86,2026-01-07-03-19-29 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,155
39,2026-01-07-09-07-00 +0000,wsj,Venezuela to Give U.S. Up to 50 Million Barrel...,https://www.wsj.com/politics/policy/venezuela-...,154


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,169,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...
48,108,2026-01-07-08-31-34 +0000,wsj,Stock Market Today: Oil Prices Fall After Trum...,https://www.wsj.com/livecoverage/stock-market-...
28,61,2026-01-07-10-00-00 +0000,wsj,The welfare-fraud scandal that led Minnesota G...,https://www.wsj.com/politics/elections/minneso...
253,55,2026-01-06-17-47-55 +0000,nypost,AI deepfakes of Nicolás Maduro flood social me...,https://nypost.com/2026/01/06/business/ai-deep...
190,53,2026-01-06-21-51-00 +0000,wsj,Secretary of State Rubio told lawmakers that r...,https://www.wsj.com/politics/national-security...
142,49,2026-01-06-23-17-54 +0000,latimes,New California tool can stop brokers from sell...,https://www.latimes.com/california/story/2026-...
210,45,2026-01-06-20-45-00 +0000,wsj,U.K. and France Agree to Set Up Military Hubs ...,https://www.wsj.com/world/europe/u-k-and-franc...
274,39,2026-01-06-16-16-08 +0000,nypost,Security clash with protesters in Iran’s Grand...,https://nypost.com/2026/01/06/world-news/secur...
318,36,2026-01-06-11-53-09 +0000,nypost,Venezuelan opposition leader declares she woul...,https://nypost.com/2026/01/06/world-news/venez...
303,29,2026-01-06-13-09-00 +0000,wsj,Next PLC Lifts Outlook as International Sales ...,https://www.wsj.com/business/retail/next-plc-l...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
